In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization

In [4]:
data_path = '/content/drive/MyDrive/ML/COOKIE/info/data/act_dataset/'

df = pd.read_csv(data_path + 'targets_ds.csv', index_col=0)

#Загрузка stft данных. Хз в каком они будут виде, так что код из этого рпздела просто для ftt данных

In [5]:
import numpy as np
import os

os.chdir(data_path + 'данные_вибрации/2024_06_01/stft_data/') #stft_data нет. ебитесь сами <3

In [9]:
vibro_time_list = []
vibro_data = []

file_names = [f for f in os.listdir() if f.endswith(".npy")]

# Извлекаем времена и данные
vibro_time_list = [f[:-4] for f in file_names]
vibro_data = [np.fromfile(f, dtype=np.float64) for f in file_names]

# Преобразуем время в формате datetime
vibro_time_list = pd.to_datetime(vibro_time_list, format="%Y-%m-%d %H-%M-%S")

KeyboardInterrupt: 

In [ ]:
vibro_times_df = pd.DataFrame(vibro_time_list, columns=['t'])
vibro_times_df = vibro_times_df.sort_values(by='t')

vibro_df = vibro_times_df
vibro_df['data'] = pd.Series(vibro_data)

In [ ]:
time_map = vibro_df.set_index('t')['data'].to_dict()

df['data'] = df['t'].map(time_map)

In [ ]:
df.drop('t', axis=1, inplace=True)
df

In [ ]:
from sklearn.model_selection import train_test_split

X = df['data']
y_1 = df['liquid_flowrate_m3day']
y_2 = df['gas_flowrate_m3day']

X_train, X_test, y_train_1, y_test_1, y_train_2, y_test_2 = train_test_split(
    X, y_1, y_2, test_size=0.2, random_state=42)

#STFT Модель

In [ ]:
def conv_block(tensor, filters, kernel_size, activation, strides=1, dropout=0.1):
  conv_layer = tf.keras.layers.Conv2D(filters=filters,
                                      kernel_size=kernel_size,
                                      strides=strides,
                                      padding='same',
                                      activation=None)(tensor)

  batch_norm_layer = tf.keras.layers.BatchNormalization()(conv_layer)
  activation_layer = tf.keras.layers.Activation(activation)(batch_norm_layer)
  dropout_layer = tf.keras.layers.Dropout(dropout)(activation_layer)

  pooling_layer = tf.keras.layers.MaxPool2D(pool_size=2,
                                            strides=2)(dropout_layer)

  return pooling_layer

def dense_block(tensor, units, activation, dropout=0.1):
  dense_layer = tf.keras.layers.Dense(units=units,
                                      activation=None)(tensor)
  batch_norm_layer = tf.keras.layers.BatchNormalization()(dense_layer)
  activation_layer = tf.keras.layers.Activation(activation)(batch_norm_layer)
  dropout_layer = tf.keras.layers.Dropout(dropout)(activation_layer)
  return dropout_layer

def get_model(input_shape=(8000, 32, 257), depth=8):      #input_shape не уверен на счет значений и их порядка
  input_layer = tf.keras.layers.Input(shape=input_shape)

  #Encoder
  for i in range(depth):
    if i == 0:
      conv = conv_block(tensor=input_layer,
                        filters=32,
                        kernel_size=3,
                        activation='relu',
                        strides=1,
                        dropout=0.1)
    else:
      conv = conv_block(tensor=conv,
                        filters=32,
                        kernel_size=3,
                        activation='relu',
                        strides=1,
                        dropout=0.1)

  #Fully-connected
  flatten_layer = tf.keras.layers.Flatten()(conv)

  dense = dense_block(tensor=flatten_layer,
                      units=100,
                      activation='relu')

  dense = dense_block(tensor=flatten_layer,
                      units=50,
                      activation='relu')

  output_layer = tf.keras.layers.Dense(units=2,
                                      activation='relu')(dense)

  model = tf.keras.models.Model(input_layer, output_layer)

  model.compile(optimizer='adam', loss='mse', metrics=['mape'])

  return model

In [ ]:
stft_model = get_model()

In [ ]:
stft_model.fit()

In [ ]:
print('Train loss:', history.history['loss'])
print('Train loss:', history.history['mape'])

In [ ]:
loss, mae, mape_value = stft_model.evaluate(X_test, {'y_1_output': y_test_1, 'y_2_output': y_test_2})

print(f'Test Loss: {loss}, MAE: {mae}, MAPE: {mape_value}')